In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用FBProphet和Vertex AI训练和部署销售预测模型

<table align="left">
   <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/sdk/SDK_FBProphet_Forecasting_Online.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
    <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/sdk/SDK_FBProphet_Forecasting_Online.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
   </td>
  <td>
<a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/sdk/SDK_FBProphet_Forecasting_Online.ipynb" target='_blank'>
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>
</table>

## 概览

本教程将指导您构建一个自定义容器来为 Vertex AI 上的 Facebook Prophet 模型提供服务。您将使用 FastAPI Python Web 服务器框架来创建一个预测端点。这个笔记本是对 [在 Vertex AI 上为 scikit-learn 模型提供服务的示例](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/sdk/SDK_Custom_Container_Prediction.ipynb) 进行修改的版本。

了解有关如何从 [testdriven.io 的文章部署和托管机器学习模型与 FastAPI 和 Heroku](https://testdriven.io/blog/fastapi-machine-learning/) 中了解有关为 FBProphet 模型提供服务的更多信息。

了解更多关于 [自定义训练](https://cloud.google.com/vertex-ai/docs/training/custom-training) 和 [Vertex AI 预测](https://cloud.google.com/vertex-ai/docs/predictions/get-predictions)。

### 目标

这份笔记本的目标是在Vertex AI上创建、部署和提供一个自定义的预测模型。这份笔记本更专注于部署模型而不是模型本身的设计。

本教程使用以下 Google Cloud ML 服务和资源：

- Vertex AI 模型注册表
- Vertex AI 端点

执行的步骤包括：
- 在本地训练一个模型，用于预测给定天数的销售额。
- 训练另一个模型，使用销售额和天气数据进行销售预测。
- 保存这两个模型。
- 构建一个FastAPI服务器，用于处理所选模型的预测。
- 使用模型工件构建一个用于提供应用程序的自定义容器映像。
- 将模型上传到Vertex AI模型注册表。
- 将模型部署到一个Vertex AI端点。
- 向部署的模型发送在线预测请求。
- 清理本次会话创建的资源。

### 数据集

本教程使用了爱荷华州在 2020 年 01 月 01 日至 2022 年 01 月 01 日期间的历史酒类销售数据，数据来源为 BigQuery 中的一个公共数据集：`bigquery-public-data:iowa_liquor_sales.sales`，并训练并部署了一个针对给定时间窗口的销售预测模型。

每个实例包括 2 个特征，一个日期时间戳和该时间段的调整后销售额。

### 成本

本教程使用Google Cloud的计费组件：

* Vertex AI
* Cloud Storage
* Artifact Registry
* Cloud Build

了解[Vertex AI定价](https://cloud.google.com/vertex-ai/pricing)，[Cloud Storage定价](https://cloud.google.com/storage/pricing)，[Artifact Registry定价](https://cloud.google.com/artifact-registry/pricing)和[Cloud Build定价](https://cloud.google.com/build/pricing)，并使用[Pricing Calculator](https://cloud.google.com/products/calculator/)根据您的预期使用量生成成本估算。

## 安装

在您的笔记本环境中安装此教程所需的软件包依赖项，例如NumPy、pystan、pandas、plotly、fbprophet、FastAPI、Uvicorn和joblib。本笔记本为每个库使用了特定版本，但通常建议使用每个软件包的最新主要GA版本。请注意，在后续步骤构建服务容器时也会使用`requirements.txt文件`。

In [ ]:
%%writefile requirements.txt
joblib==1.1.0
google-cloud-storage==2.4.0
google-cloud-aiplatform==1.16.0
google-cloud-bigquery==2.34.4
numpy
pandas==2.0.3
plotly==5.9.0
pyarrow==8.0.0
fastapi~=0.63
pystan
'shapely<2'
prophet==1.1.4

In [ ]:
# install dependencies from requirements.txt
! pip3 install -r requirements.txt -q

### 仅限Colab：取消注释以下单元格以重启内核

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## 开始之前

### 设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下操作：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 查看支持页面：[查找项目ID](https://support.google.com/googleapi/answer/7014113)。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

#### 区域

您还可以更改 Vertex AI 使用的 `REGION` 变量。了解有关 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations) 的更多信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

### 验证你的谷歌云账户

根据你的Jupyter环境，你可能需要手动进行身份验证。请按照下面的相关说明进行操作。

**1. Vertex AI Workbench**
* 无需操作，因为你已经验证过了。

**2. 本地JupyterLab 实例，取消注释并运行:**

In [ ]:
# ! gcloud auth login

3. 协作、取消注释并运行:

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

请查看如何在https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples为您的服务帐号授予Cloud Storage权限。

创建一个云存储桶

创建一个存储桶来存储中间工件，比如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有在您的存储桶尚不存在时才需要运行以下单元格来创建您的云存储存储桶。

In [ ]:
! gsutil mb -l {REGION} {BUCKET_URI}

### 为您的模型产物创建一个本地目录

In [ ]:
%mkdir -p app

### 配置模型、存储库和构件名称

针对模型配置以下参数：

- `MODEL_ARTIFACT_DIR` - 模型构件在云存储存储桶中的文件夹目录路径，例如："my-models/fraud-detection/trial-4"。
- `REPOSITORY` - 要创建或使用的构件存储库的名称。
- `IMAGE` - 推送的容器镜像的名称。
- `MODEL_NAME` - 模型的名称。
- `MODEL_DISPLAY_NAME` - 模型的显示名称。

In [ ]:
MODEL_ARTIFACT_DIR = "[your-model-artifact-dir-name]"  # @param {type:"string"}
REPOSITORY = "[your-repository-name]"  # @param {type:"string"}
IMAGE = "[your-image-name]"  # @param {type:"string"}
MODEL_NAME = "[your-model-name]"  # @param {type:"string"}
MODEL_DISPLAY_NAME = "[your-model-display-name]"  # @param {type:"string"}

In [ ]:
# Otherwise, choose deafult names
if MODEL_ARTIFACT_DIR == "[your-model-artifact-dir-name]":
    MODEL_ARTIFACT_DIR = "custom-fb-container-prediction-model"
if REPOSITORY == "[your-repository-name]":
    REPOSITORY = "custom-fb-container-prediction"
if IMAGE == "[your-image-name]":
    IMAGE = "custom-fb-fastapi-server"
if MODEL_NAME == "[your-model-name]":
    MODEL_NAME = "fb_custom_container"
if MODEL_DISPLAY_NAME == "[your-model-display-name]":
    MODEL_DISPLAY_NAME = "FB Prophet Forecast"

### 导入库

In [ ]:
import os

import google.auth
import joblib
from google.cloud import aiplatform, bigquery
from prophet import Prophet

初始化BigQuery客户端和Python的Vertex AI SDK。

In [ ]:
credentials, _ = google.auth.default(
    scopes=["https://www.googleapis.com/auth/cloud-platform"]
)
bqclient = bigquery.Client(
    credentials=credentials,
    project=PROJECT_ID,
)

aiplatform.init(project=PROJECT_ID, location=REGION)

## 训练和保存模型
接下来，您将定义使用fbprophet训练时间序列模型的函数，针对一些销售类别。

在接下来的步骤中定义了两种类型的预测函数。一种只使用销售数据，另一种使用[天气变量作为额外的回归器](https://facebook.github.io/prophet/docs/seasonality,_holiday_effects,_and_regressors.html)进行预测。

天气数据来自BigQuery中的一个公共数据集。了解更多关于[天气数据](https://cloud.google.com/blog/products/gcp/global-historical-daily-weather-data-now-available-in-bigquery)的信息。

最后，这些函数将使用joblib将您训练好的模型导出为一个（`.sav`）文件。

### 定义销售模型训练函数

在接下来的单元格中，您将定义一个函数，用于从BigQuery中获取酒类销售数据，处理数据并训练和保存预测模型。

In [ ]:
def train(category):
    # Download query results. # change to Data, Adj_Close
    query_string = f"""
    SELECT date as Date, upper(category_name) as category_name, sum(sale_dollars) as sales
    FROM `bigquery-public-data.iowa_liquor_sales.sales`
    where upper(category_name) = '{category}'
      and date >= '2020-01-01' 
      and date < '2022-01-01'
    group by Date, category_name
    """

    data = bqclient.query(query_string).result().to_dataframe()
    data.plot(title=f"{category} Daily Sales", x="Date", y="sales")

    # Copy results into a dataframe with two columns ds=Date, y=value
    df_forecast = data.copy()
    df_forecast.reset_index(inplace=True)
    df_forecast["ds"] = df_forecast["Date"]
    df_forecast["y"] = df_forecast[
        "sales"
    ]  # Add underscore as BQ changes this field name
    df_forecast = df_forecast[["ds", "y"]]

    # Train the Prophet model
    model = Prophet()
    model.fit(df_forecast)

    # Save the model locally
    joblib.dump(model, "{}.sav".format(category))

### 为两个样本类别训练和保存模型

运行训练函数，为 *加拿大威士忌* 和 *美国伏特加* 生成销售预测模型。

In [ ]:
%cd app/

# Train a sales forecast model for CANADIAN WHISKIES
train("CANADIAN WHISKIES")
# Train a sales forecast model for AMERICAN VODKAS
train("AMERICAN VODKAS")

%cd ..

### 为销售和天气模型定义训练函数

创建一个考虑天气数据的训练函数。用于获取天气数据的SQL查询从BigQuery的公共天气数据集`bigquery-public-data.ghcn_d`中检索了两年的数据。该查询使用移动平均值生成了100个未来的天气数据点。

在获取天气数据之后，它被保存在`temp`列中，并作为额外的回归器传递给预测模型。

了解更多关于[FBProphet回归器](https://facebook.github.io/prophet/docs/seasonality,_holiday_effects,_and_regressors.html)。

In [ ]:
def train_w_regressor(category):
    # Download query results. # change to Data, Adj_Close
    query_string = """
    SELECT date as Date, upper(category_name) as category_name, sum(sale_dollars) as sales
    FROM `bigquery-public-data.iowa_liquor_sales.sales`
    where upper(category_name) = '{CATEGORY}'
      and date >= '2020-01-01'
      and date < '2022-01-01'
    group by Date, category_name
    """.format(
        CATEGORY=category
    )

    query_string_regressor = """
    WITH history as
    (
    SELECT
      wx.date as ds,
      AVG(wx.value/10) AS temp
    FROM
      `bigquery-public-data.ghcn_d.ghcnd_stations` AS stn
    JOIN
      `bigquery-public-data.ghcn_d.ghcnd_2021` AS wx ON wx.id = stn.id
    WHERE
      stn.state = 'AZ'
      AND wx.element = 'TMIN'
      AND wx.qflag IS NULL
    GROUP by wx.date
    union all
    SELECT
      wx.date as ds,
      AVG(wx.value/10) AS temp
    FROM
      `bigquery-public-data.ghcn_d.ghcnd_stations` AS stn
    JOIN
      `bigquery-public-data.ghcn_d.ghcnd_2020` AS wx ON wx.id = stn.id
    WHERE
      stn.state = 'AZ'
      AND wx.element = 'TMIN'
      AND wx.qflag IS NULL
    GROUP by wx.date
    )
    ,next_100_days as
    (
    SELECT DATE_ADD(max(stn.date) over (order by 1), INTERVAL row_number() over (order by stn.id) DAY) as ds, null as temp
    FROM `bigquery-public-data.ghcn_d.ghcnd_2021` AS stn
    LIMIT 100
    )
    ,combined_data as
    (
    select *
    from history
    union all
    select * from next_100_days
    )
    select c.ds
          ,case when temp is null then AVG(c.temp) OVER (ORDER BY c.ds ASC ROWS 100 PRECEDING)
           else temp end as temp
    from combined_data c;
    """

    data = bqclient.query(query_string).result().to_dataframe()
    data.plot(title=f"{category} Daily Sales", x="Date", y="sales")

    # Copy results into a dataframe with two columns ds=Date, y=value
    df_forecast = data.copy()
    df_forecast.reset_index(inplace=True)
    df_forecast["ds"] = df_forecast["Date"]
    df_forecast["y"] = df_forecast[
        "sales"
    ]  # Add underscore as BQ changes this field name
    df_forecast = df_forecast[["ds", "y"]]

    # Return the weather data from BQ
    data_regressor = bqclient.query(query_string_regressor).result().to_dataframe()
    df_regressor = data_regressor.copy()

    # Deal with any NaN values using fillna
    df_regressor_out = df_regressor.fillna(method="pad")
    df_regressor_out.to_csv(f"{category}_weather.csv", index=False)

    # Add a temp column to the df_forecast dataframe, and populate it from the df_regressor dataframe
    df_forecast["temp"] = df_forecast.ds.map(
        df_regressor.set_index("ds")["temp"].to_dict()
    )

    df_forecast.loc[:, "temp"] = df_forecast.fillna(method="pad").fillna(
        method="backfill"
    )  # Address NaN values of temp

    # Train the Prophet model with the regressor
    model = Prophet()
    model = model.add_regressor("temp")
    model.fit(df_forecast)

    # Save the model locally, with _weather appended to the file name
    joblib.dump(model, f"{category}_weather.sav")

使用天气数据对两个样本类别进行模型训练和保存

对*加拿大威士忌*和*美国伏特加*运行训练函数，并将模型保存在本地。

In [ ]:
%cd app/

# Train a sales forecast model(with weather data) for CANADIAN WHISKIES
train_w_regressor("CANADIAN WHISKIES")
# Train a sales forecast model(with weather data) for AMERICAN VODKAS
train_w_regressor("AMERICAN VODKAS")

%cd ..

### 将模型工件和自定义代码上传到云存储

在部署模型进行服务之前，Vertex AI 需要访问云存储中的以下文件：

* 对于您想要检索预测的每个销售类别，需要 `{category}.sav`（模型工件）。
* `*.csv` - 在预测时需要天气回归器的数据。

运行以下单元格将您的文件上传到亲亲云存储。

In [ ]:
%cd app
!gsutil cp *.sav *.csv {BUCKET_URI}/{MODEL_ARTIFACT_DIR}/
%cd ..

## 本地模型测试

在下面的单元格中测试训练好的预测模型（包括具有和不具有天气回归器的模型）。

设置环境变量。

### 测试销售模型

In [ ]:
%cd app

import datetime

import pandas as pd
from google.cloud import storage

gcs_client = storage.Client(project=PROJECT_ID)

# Set a sample category and number of days to test with
category = "CANADIAN WHISKIES"
days = 7

# Load the sales model
fname = f"{category}.sav"
model = joblib.load(f"{category}.sav")

# Create a dataframe that ranges from 2020 to Today + the number of days set above
TODAY = datetime.date.today()
future = TODAY + datetime.timedelta(days=days)

dates = pd.date_range(
    start="2020-01-01",
    end=future.strftime("%m/%d/%Y"),
)
df = pd.DataFrame({"ds": dates})

# Run a prediction for these dates and save it to a forecast dataframe
forecast = model.predict(df)

model.plot(forecast).savefig(f"{category}_plot.png")
model.plot_components(forecast).savefig(f"{category}_plot_components.png")

forecast.tail(days).to_dict("records")

%cd ..

### 测试销售和天气模型

In [ ]:
%cd app

gcs_client = storage.Client(project=PROJECT_ID)

# Set a sample category and number of days to test with
category = "CANADIAN WHISKIES_weather"
days = 7

# Load the sales and weather model
fname = f"{category}.sav"
model = joblib.load(f"{category}.sav")

# Load the data
fname_csv = f"{category}.csv"
df_regressor = pd.read_csv(fname_csv)

# Return the data from the regressor that ranges from 2020 to Today + the number of days set above
TODAY = datetime.date.today()
future = TODAY + datetime.timedelta(days=days)

start = "2020-01-01"
after_start_date = df_regressor["ds"] >= start
before_end_date = df["ds"] <= future.strftime("%m/%d/%Y")
between_two_dates = after_start_date & before_end_date
df_final = df_regressor.loc[between_two_dates]
df_final.tail()

# Run a prediction for these dates and the regressor
forecast = model.predict(df_final)

model.plot(forecast).savefig(f"{category}_plot.png")
model.plot_components(forecast).savefig(f"{category}_plot_components.png")

%cd ..

构建一个FastAPI服务器

为了服务于这两个模型的预测结果，构建一个FastAPI服务器应用程序。

In [ ]:
%%writefile app/main.py
from fastapi import FastAPI, Request
from fastapi.exceptions import RequestValidationError
from fastapi.responses import JSONResponse

import joblib
import json
import numpy as np
import os
import datetime
import pandas as pd
import errno

from google.cloud import storage

app = FastAPI()
gcs_client = storage.Client()

@app.exception_handler(Exception)
async def validation_exception_handler(request, err):
    base_error_message = f"Failed to execute: {request.method}: {request.url}"
    # Change here to LOGGER
    return JSONResponse(status_code=400, content={"message": f"{base_error_message}. Detail: {err}"})

    
def download_model(category="CANADIAN WHISKIES", regressor=""): #defaults to "CANADIAN WHISKIES"
    if not category:
        raise HTTPException(status_code=400, detail=f"category not found. category={category}")
    fname = f"{category}{regressor}.sav"
    with open(f"{fname}", 'wb') as model_g:
        gcs_client.download_blob_to_file(
            f"{os.environ['AIP_STORAGE_URI']}/{fname}", model_g
        )
    model = joblib.load (fname)
    return model
    
def predict_in(model, days=7, regressor="", category="CANADIAN WHISKIES"):
    TODAY = datetime.date.today()
    future = TODAY + datetime.timedelta(days=days)

    dates = pd.date_range(start="2020-01-01", end=future.strftime("%m/%d/%Y"),)
    df = pd.DataFrame({"ds": dates})

    if (regressor > ""):
        fname_csv = f"{category}{regressor}.csv"
        with open(f"{fname_csv}", 'wb') as model_csv:
            gcs_client.download_blob_to_file(
                f"{os.environ['AIP_STORAGE_URI']}/{fname_csv}", model_csv
            )

        df_regressor = pd.read_csv(fname_csv)
        start="2020-01-01"
        after_start_date = df_regressor["ds"] >= start
        before_end_date = df["ds"] <= future.strftime("%m/%d/%Y")
        between_two_dates = after_start_date & before_end_date
        df_final = df_regressor.loc[between_two_dates]
        df_final.tail()
        
        forecast = model.predict(df_final)
    
    else:
        forecast = model.predict(df)

    model.plot(forecast).savefig("model_plot.png")
    model.plot_components(forecast).savefig("model_plot_components.png")    
    
    return forecast.tail(days).to_dict("records")


def convert(prediction_list):
    output = {}
    for data in prediction_list:
        date = data["ds"].strftime("%m/%d/%Y")
        output[date] = data["trend"]
    return output


@app.get(os.environ['AIP_HEALTH_ROUTE'], status_code=200)
def health():
    return {}


@app.post(os.environ['AIP_PREDICT_ROUTE'])
async def predict(request: Request):
    body = await request.json()
    try:
        if type(body) is list:
            body = body[0]
    except:
        None #Do Nothing
    print (body)
    instances = body["instances"]

    try:
        if type(instances) is list:
            instances = instances[0]
    except:
        None #Do Nothing
    print(instances)    
    category = instances["category"] 
    days = instances["days"]
    regressor = instances["regressor"]
    
    try:
        if type(category) is list:
            category = category[0]
    except:
        None #Do Nothing
    print(category) 

    try:
        if type(days) is list:
            days = days[0]
    except:
        None #Do Nothing
    print(days) 

    try:
        if type(regressor) is list:
            regressor = regressor[0]
    except:
        None #Do Nothing
    print(regressor) 
    
    model_download = download_model (category, regressor)
    prediction_list = predict_in(model_download, days, regressor, category)  
    
    if not prediction_list:
        raise HTTPException(status_code=400, detail="Model not found.")
    
    prediction_output = convert(prediction_list)
    final_output = [(k, v) for k, v in prediction_output.items()] 
    
    return {"predictions": final_output}

### 添加预启动脚本
FastAPI 在启动服务器之前执行此脚本。为了在与 Vertex AI 预期相同的端口上运行 FastAPI，将 `PORT` 环境变量设置为等于 `AIP_HTTP_PORT`。

In [ ]:
%%writefile app/prestart.sh
export PORT=$AIP_HTTP_PORT

### 创建测试实例
要了解有关在 JSON 中格式化输入实例的更多信息，请[阅读文档。](https://cloud.google.com/ai-platform-unified/docs/predictions/online-predictions-custom-models#request-body-details)

In [ ]:
%%writefile instances.json
{"instances": [{"category":["CANADIAN WHISKIES"], "days": [30], "regressor": ["_weather"]}]}

构建并推送容器至 Artifact Registry

使用`tiangolo/uvicorn-gunicorn-fastapi`作为基础镜像编写`Dockerfile`。这将自动使用Gunicorn和Uvicorn为您运行FastAPI。

了解更多关于[使用Docker部署FastAPI服务器](https://fastapi.tiangolo.com/deployment/docker/)。

In [ ]:
%%writefile Dockerfile

FROM tiangolo/uvicorn-gunicorn-fastapi:python3.9

COPY ./app /app
COPY requirements.txt requirements.txt

RUN pip install -r requirements.txt

### 本地构建镜像（可选）

使用docker在本地构建镜像。日志存储在`logs.txt`中。

**注意：** Docker仅用于在本地测试容器。用于部署到Artifact注册表的是Cloud-Build。

In [ ]:
import sys


def build_image():
    ! sudo docker build --tag="{REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE}" . > logs.txt


IS_COLAB = "google.colab" in sys.modules
if not IS_COLAB and not os.getenv("IS_TESTING"):
    print("Building the image...")
    build_image()
    print("Process completed !")

### 在本地运行并测试容器（可选）

在后台模式下测试在本地运行容器，并提供容器所需的环境变量。这些环境变量在容器部署到 Vertex AI 后由 Vertex AI Predictions 提供。测试 `/health` 和 `/predict` 路由，然后停止运行的镜像。

**注意**: 本节测试容器的本地运行。但是，如果底层环境是Colab，则`docker`命令会被自动跳过。

In [ ]:
if not IS_COLAB and not os.getenv("IS_TESTING"):
    ! sudo docker stop local-ts
    ! sudo docker rm local-ts
    ! sudo docker run -d -p 80:8080 \
        --name=local-ts \
        -e AIP_HTTP_PORT=8080 \
        -e AIP_HEALTH_ROUTE=/health \
        -e AIP_PREDICT_ROUTE=/predict \
        -e AIP_STORAGE_URI={BUCKET_URI}/{MODEL_ARTIFACT_DIR} \
        "{REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE}"

评估健康路线。

In [ ]:
if not IS_COLAB and not os.getenv("IS_TESTING"):
    ! curl localhost/health

### 测试预测路线（可选）

传递 `instances.json` 并测试预测路线。

In [ ]:
if not IS_COLAB and not os.getenv("IS_TESTING"):
    ! curl -X POST \
      -d @instances.json \
      -H "Content-Type: application/json; charset=utf-8" \
      localhost/predict

现在，存储一个没有`regressor`的实例，并测试预测路线。这将从训练模型中获取没有天气回归变量的预测。

In [ ]:
%%writefile instances.json
{"instances": [{"category":["CANADIAN WHISKIES"], "days": [30], "regressor": [""]}]}

In [ ]:
if not IS_COLAB and not os.getenv("IS_TESTING"):
    ! curl -X POST \
      -d @instances.json \
      -H "Content-Type: application/json; charset=utf-8" \
      localhost/predict

停止并在本地删除容器。

In [ ]:
if not IS_COLAB and not os.getenv("IS_TESTING"):
    ! sudo docker stop local-ts
    ! sudo docker rm local-ts

#### 启用 Artifact Registry API
您必须启用项目的 Artifact Registry API 服务。

<a href="https://cloud.google.com/artifact-registry/docs/enable-service">了解更多关于启用服务的信息</a>。

In [ ]:
! gcloud services enable artifactregistry.googleapis.com

if os.getenv("IS_TESTING"):
    ! sudo apt-get update --yes && sudo apt-get --only-upgrade --yes install google-cloud-sdk-cloud-run-proxy google-cloud-sdk-harbourbridge google-cloud-sdk-cbt google-cloud-sdk-gke-gcloud-auth-plugin google-cloud-sdk-kpt google-cloud-sdk-local-extract google-cloud-sdk-minikube google-cloud-sdk-app-engine-java google-cloud-sdk-app-engine-go google-cloud-sdk-app-engine-python google-cloud-sdk-spanner-emulator google-cloud-sdk-bigtable-emulator google-cloud-sdk-nomos google-cloud-sdk-package-go-module google-cloud-sdk-firestore-emulator kubectl google-cloud-sdk-datastore-emulator google-cloud-sdk-app-engine-python-extras google-cloud-sdk-cloud-build-local google-cloud-sdk-kubectl-oidc google-cloud-sdk-anthos-auth google-cloud-sdk-app-engine-grpc google-cloud-sdk-pubsub-emulator google-cloud-sdk-datalab google-cloud-sdk-skaffold google-cloud-sdk google-cloud-sdk-terraform-tools google-cloud-sdk-config-connector
    ! gcloud components update --quiet

### 创建一个私有的Docker仓库
您的第一步是在Google Artifact Registry中创建自己的Docker仓库。

1 - 运行gcloud artifacts repositories create命令，使用您的地区和描述“docker仓库”创建一个新的Docker仓库。

2 - 运行gcloud artifacts repositories list命令来验证您的仓库是否已创建。

In [ ]:
! gcloud artifacts repositories create {REPOSITORY} --repository-format=docker --location={REGION} --description="Docker repository"

! gcloud artifacts repositories list

使用Cloud-Build将图像推送到创建的存储库。

In [ ]:
!gcloud builds submit --region={REGION} --tag={REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE}

使用 Python SDK 从 artifact URI 上载并部署您的模型，创建一个 Vertex AI 模型。

In [ ]:
model = aiplatform.Model.upload(
    display_name=MODEL_DISPLAY_NAME,
    artifact_uri=f"{BUCKET_URI}/{MODEL_ARTIFACT_DIR}",
    serving_container_image_uri=f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE}",
)

将模型部署到Vertex AI终端

将模型部署到终端。完成此步骤后，模型将被部署并准备好进行在线预测。

In [ ]:
endpoint = model.deploy(machine_type="n1-standard-4")

## 请求预测

向部署到端点的模型发送在线请求并获得预测结果。

### 使用Python SDK

使用Python SDK从端点获取预测结果。

In [ ]:
# Send an instance for the model without regressor
endpoint.predict(
    instances=[{"category": ["CANADIAN WHISKIES"], "days": [30], "regressor": [""]}]
)

In [ ]:
# Send an instance for the model with regressor
endpoint.predict(
    instances=[
        {"category": ["CANADIAN WHISKIES"], "days": [30], "regressor": ["_weather"]}
    ]
)

使用REST

使用curl请求从端点获取预测。

In [ ]:
ENDPOINT_ID = endpoint.name

In [ ]:
! curl \
-H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
-d @instances.json \
https://{REGION}-aiplatform.googleapis.com/v1/projects/{PROJECT_ID}/locations/{REGION}/endpoints/{ENDPOINT_ID}:predict

使用gcloud CLI获得通过终端点的预测。

In [ ]:
!gcloud ai endpoints predict $ENDPOINT_ID \
  --region=$REGION \
  --json-request=instances.json

清理

要清理此项目中使用的所有Google Cloud资源，您可以[删除用于教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在此教程中创建的各个资源：

- 模型
- 端点
- Artefact Registry镜像
- Artefact Repository：将`delete_art_repo`设置为**True**以删除在此教程中创建的存储库。
- Cloud Storage存储桶：将`delete_bucket`设置为**True**以删除在此教程中使用的Cloud Storage存储桶。

In [ ]:
delete_bucket = False
delete_art_repo = False

# # Undeploy model and delete endpoint
endpoint.undeploy_all()
endpoint.delete()

# # Delete the model resource
model.delete()

# Delete the container image from Artifact Registry
!gcloud artifacts docker images delete \
    --quiet \
    --delete-tags \
    {REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE}

# # Delete the artifact registry
if delete_art_repo or os.getenv("IS_TESTING"):
    ! gcloud artifacts repositories delete {REPOSITORY} --location=$REGION -q

# Delete the Cloud Storage bucket
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI